# PART 21: Setup & Data Loading

In [1]:
# Input: data/processed/smart_routing.csv
# Outputs:
# - visualizations/priority_map.html
# - visualizations/priority_table.csv
# Note: We use CircleMarker to minimize Icon/Font dependency.
#   Marker size is dynamic based on priority_score (B approach) + small bonus for high risk.

from pathlib import Path
import pandas as pd
import numpy as np
import folium
from folium import Map, FeatureGroup, CircleMarker
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from branca.element import MacroElement
from jinja2 import Template

In [2]:
# 0) Paths and files
ROOT = Path("..")  # notebook is defaulted under 'notebooks/'
DATA_DIR = ROOT / "data" / "processed"
VIZ_DIR  = ROOT / "visualizations"
VIZ_DIR.mkdir(parents=True, exist_ok=True)

IN_ROUTE = DATA_DIR / "smart_routing.csv"
OUT_HTML = VIZ_DIR / "priority_map.html"
OUT_TAB  = VIZ_DIR / "priority_table.csv"

In [3]:
# 1) Data loading and minimum checks
df = pd.read_csv(IN_ROUTE)
min_cols = [
    "Order_ID","y_pred","risk_level","priority_score",
    "Drop_Latitude","Drop_Longitude"
]
missing = [c for c in min_cols if c not in df.columns]
if missing:
    raise KeyError(f"Missing column(s) in smart_routing.csv: {missing}")

In [4]:
# Type/format securing
df['Drop_Latitude']  = pd.to_numeric(df['Drop_Latitude'], errors='coerce')
df['Drop_Longitude'] = pd.to_numeric(df['Drop_Longitude'], errors='coerce')
df['risk_level'] = df['risk_level'].astype(str).str.lower()

# Coordinate range filter (exclude invalid values)
base = df[df['Drop_Latitude'].between(-90, 90) & df['Drop_Longitude'].between(-180, 180)].copy()
if base.empty:
    raise ValueError("Valid coordinates are not available; a map cannot be generated.")

print("Number of records:", len(df))
print("Number of records with coordinates:", len(base))
print(base['risk_level'].value_counts())

Number of records: 8703
Number of records with coordinates: 8703
low       4456
medium    2773
high      1474
Name: risk_level, dtype: int64


# PART 22: Map Setups

In [5]:
#2) Colors + Dynamic Radius (B approach)
RISK_COLOR = {"low":"green","medium":"orange","high":"red"}

# 5–95 percentiles for robust scaling
p5, p95 = np.percentile(base['priority_score'], [5, 95])

def scale_radius(score: float, rmin: float = 4.0, rmax: float = 12.0) -> float:
    """Scale priority_score to radii [rmin, rmax] within the range [p5, p95]."""
    if pd.isna(score):
        return (rmin + rmax) / 2
    if p95 == p5:
        return (rmin + rmax) / 2
    s = (score - p5) / (p95 - p5)
    s = max(0.0, min(1.0, s))
    return rmin + s * (rmax - rmin)

HIGH_BONUS = 2.0  # Small bonus for high risk

In [6]:
#3) Maps & Layers
m = folium.Map(location=[base['Drop_Latitude'].mean(), base['Drop_Longitude'].mean()],
               zoom_start=11, control_scale=True)

layer_high  = folium.FeatureGroup(name="High Risk (markers)", show=True)   # Just start with high enabled
layer_medlo = folium.FeatureGroup(name="Medium/Low (heatmap)", show=True)  # heatmap

In [7]:
# 4a) High risk: scaled CircleMarker (our B approach)
RISK_COLOR = {"low":"green","medium":"orange","high":"red"}
p5, p95 = np.percentile(base['priority_score'], [5, 95])
def scale_radius(score, rmin=4, rmax=12):
    if p95 == p5: return (rmin+rmax)/2
    s = (score - p5)/(p95 - p5); s = max(0, min(1, s))
    return rmin + s*(rmax-rmin)

# Draw the high points LAST (keep them on top)
high = base[base['risk_level'].str.lower() == 'high'].copy()
bounds = []

for _, row in high.iterrows():
    lat, lon = float(row['Drop_Latitude']), float(row['Drop_Longitude'])
    rad = scale_radius(row.get('priority_score', 0.0)) + 2  # high bonus
    popup_html = "<br>".join(
        f"<b>{k}:</b> {row[k]}" for k in
        ['Order_ID','risk_level','priority_score','y_pred','Vehicle','Traffic','Order_Period','Area','Category']
        if k in high.columns and pd.notna(row.get(k))
    )
    folium.CircleMarker(
        location=[lat, lon],
        radius=float(rad),
        color=RISK_COLOR['high'],
        weight=2, fill=True, fill_opacity=0.9,
        tooltip=f"{row.get('Order_ID','')} | high | score={row.get('priority_score',0):.2f}",
        popup=folium.Popup(popup_html, max_width=350)
    ).add_to(layer_high)
    bounds.append((lat, lon))

In [8]:
# 4b) Medium/Low: Specify intensity via heatmap (priority_score weighting)
medlo = base[base['risk_level'].str.lower().isin(['medium','low'])].copy()
heat_data = medlo[['Drop_Latitude','Drop_Longitude','priority_score']].values.tolist()
HeatMap(heat_data, name="Density (medium/low)", radius=18, blur=24, max_zoom=12).add_to(layer_medlo)

In [9]:
#5) Additions
m.add_child(layer_high); m.add_child(layer_medlo)
folium.LayerControl().add_to(m)
if bounds:
    m.fit_bounds(bounds, padding=(20, 20))

# PART 23: Setups for HTML

In [10]:
# 6) Basit legend (HTML)
legend_html = '''
<div style="position: fixed; bottom: 30px; left: 30px; z-index: 9999; background: white; padding: 10px 12px; border:1px solid #bbb; border-radius:8px;">
  <b>Legend</b><br>
  <span style="color: white; background: red; padding:2px 6px; border-radius:4px;">high</span>
  <span style="color: white; background: orange; padding:2px 6px; border-radius:4px;">medium</span>
  <span style="color: white; background: green; padding:2px 6px; border-radius:4px;">low</span>
</div>
'''
class FloatLegend(MacroElement):
    def __init__(self, html):
        super().__init__()
        self._template = Template(f"{{% macro html(this, kwargs) %}}{html}{{% endmacro %}}")

m.get_root().add_child(FloatLegend(legend_html))

In [11]:
#7) Save
m.save(str(OUT_HTML))
print("Saved:")
print(" - Map:", OUT_HTML)
print(" - Table:", OUT_TAB)

Saved:
 - Map: ..\visualizations\priority_map.html
 - Table: ..\visualizations\priority_table.csv
